In [1]:
import pandas as pd
import numpy as np
import yaml
import math
import os
import sys

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
config_filename = "C:/Projects/SANDAG Calculators/carshare/carshare/data/config.yml"
#config_filename = "D:/Projects/SANDAG/client_sandag_off_model_calculators/carshare/data/config_AK.yml"

In [4]:
def get_adult_population (person_df, houshold_df):
    # compute adult population by mgra
    
    adults_df = person_df.query("age > 15 and age < 66", engine = "python")
    adults_df = pd.merge(adults_df, houshold_df, on = "hhid", how = "left")
    adults_mgra_df = adults_df.groupby(["mgra"])["hhid"].count().reset_index(name = 'adult_pop')
    
    return adults_mgra_df

In [5]:
def get_emission_factors(input_emission_data, scen_year):
    # prepare a dataframe of emission factors for the scen year
    
    emission_df = input_emission_data[input_emission_data["Year"] == scen_year]
    emission_df = emission_df[emission_df["Vehicle Type"] == "Passenger Car"]
    emission_df.reset_index(inplace = True, drop = True)

    co2_runex_emission_factor = emission_df["CO2 RunEx Emission Factor (gr/mile)"][0]
    co2_strex_emission_factor = emission_df["CO2 StrEx Emission Factor (gr/trip)"][0]

    out_data = {
        "Variable": ["Year",
                     "CO2 RunEx Emission Factor (gr/mile)",
                     "CO2 StrEx Emission Factor (gr/trip)"],
        "Value": [scen_year,
                  co2_runex_emission_factor,
                  co2_strex_emission_factor]
    }

    out_df = pd.DataFrame(out_data)

    return out_df

In [6]:
def compute_regional_reductions(regional_careshare_market_df, emission_factors, carshare_vmt_reduction_factor, scenario_year):
    # calculate regional vmt and emission reduction from carshare

    co2_runex_emission_factor = emission_factors.loc[
        emission_factors.Variable == "CO2 RunEx Emission Factor (gr/mile)", "Value"].values[0]

    GRAMS_TO_SHORT_TONS = 0.0000011

    total_carshare_market = regional_careshare_market_df["total_carshare_market"].sum() 
    
    regional_population = regional_careshare_market_df["pop"].sum()

    total_carshare_vmt_reduction = total_carshare_market * carshare_vmt_reduction_factor

    ghg_reduction = total_carshare_vmt_reduction * co2_runex_emission_factor * GRAMS_TO_SHORT_TONS

    daily_ghg_per_capita_reduction = ghg_reduction * 2000  / regional_population 

    out_data = {
        "Variable": ["Regional Population",
                     "Total Carshare Market",
                     "Total VMT Reduction by Carsharing",
                     "Daily Total GHG Reduction (short tons)",
                     "Daily Per Capita GHG Reduction (lbs/person)"],
        "Value": [regional_population,
                  total_carshare_market,
                  total_carshare_vmt_reduction,
                  ghg_reduction,
                  daily_ghg_per_capita_reduction]
    }

    out_df = pd.DataFrame(out_data)
    
    return out_df

In [7]:
def compute_corridor_reductions(corridor_carshare_market_df, emission_factors, carshare_vmt_reduction_factor, scenario_year):
    # calculate vmt and emission reductions from carshare by corridor
    
    co2_runex_emission_factor = emission_factors.loc[
        emission_factors_df.Variable == "CO2 RunEx Emission Factor (gr/mile)", "Value"].values[0]

    GRAMS_TO_SHORT_TONS = 0.0000011

    work_df = corridor_carshare_market_df.copy()

    work_df["Scenario Year"] = scenario_year
    
    work_df["Total Daily VMT Reduction"] = work_df["total_carshare_market"] * carshare_vmt_reduction_factor

    work_df["Daily Total GHG Reduction (short tons)"] = work_df["Total Daily VMT Reduction"] * co2_runex_emission_factor * GRAMS_TO_SHORT_TONS
    
    work_df.rename(columns={"total_carshare_market" : "Total Carshare Market"},  inplace=True)

    work_df.set_index('corridor', inplace=True)

    work_df = work_df[["Scenario Year", "Total Carshare Market", "Total Daily VMT Reduction", "Daily Total GHG Reduction (short tons)"]]

    out_df = work_df.transpose()
    out_df.reset_index(inplace = True)
    out_df.rename(columns = {"index": "Variable"}, inplace = True)

    return out_df

In [8]:
def compute_market_segments_reductions(corridor_carshare_market_df, emission_factors, carshare_vmt_reduction_factor, scenario_year):
    # calculate vmt and emission reductions for market segements
    co2_runex_emission_factor = emission_factors.loc[
        emission_factors_df.Variable == "CO2 RunEx Emission Factor (gr/mile)", "Value"].values[0]

    GRAMS_TO_SHORT_TONS = 0.0000011

    work_df = corridor_carshare_market_df.copy()

    work_df["Scenario Year"] = scenario_year

    work_df["Total daily VMT reduction general population market"] = work_df["mobility_hub"] * carshare_vmt_reduction_factor

    work_df["Total daily VMT reduction college staff market"] = work_df["college_staff"] * carshare_vmt_reduction_factor

    work_df["Total daily VMT reduction college students market"] = work_df["college_student"] * carshare_vmt_reduction_factor

    work_df["Total daily VMT reduction military market"] = work_df["military_base"] * carshare_vmt_reduction_factor


    work_df["Daily Total GHG reduction general population market (short tons)"] = work_df["Total daily VMT reduction general population market"] * co2_runex_emission_factor * GRAMS_TO_SHORT_TONS

    work_df["Daily Total GHG reduction college staff market (short tons)"] = work_df["Total daily VMT reduction college staff market"] * co2_runex_emission_factor * GRAMS_TO_SHORT_TONS

    work_df["Daily Total GHG reduction college students market (short tons)"] = work_df["Total daily VMT reduction college students market"] * co2_runex_emission_factor * GRAMS_TO_SHORT_TONS

    work_df["Daily Total GHG reduction military market (short tons)"] = work_df["Total daily VMT reduction military market"] * co2_runex_emission_factor * GRAMS_TO_SHORT_TONS

    work_df.rename(columns = {"mobility_hub" : "general population market", "college_staff" : "college staff market", 
                            "college_student": "college student market", "military_base" : "military market"},  inplace=True)

    work_df.set_index('corridor', inplace = True)

    work_df.drop(["total_carshare_market"], axis = 1, inplace = True)
    work_df.insert(0, "Scenario Year", work_df.pop("Scenario Year"))
    
    out_df = work_df.transpose()
    out_df.reset_index(inplace = True)
    out_df.rename(columns = {"index": "Variable"}, inplace = True)

    return out_df

In [9]:
def write_results(results_dict, out_file_name, out_dir):
    with pd.ExcelWriter(os.path.join(out_dir, out_file_name)) as writer:
        for key, value in results_dict.items():
            value.to_excel(writer, sheet_name = key, index = False)

In [10]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [11]:
# read config: inputs
mgra_scen_input_file = config['inputs']['mgra_scen_input_file']
mgra_base_input_file = config['inputs']['mgra_base_input_file']
household_input_file = config['inputs']['household_input_file']
person_input_file = config['inputs']['person_input_file']
geography_xwalk_file = config['inputs']['geography_xwalk_file']
emission_factors_file = config['inputs']['emission_factors_file']
corridor_mgra_xwalk_file = config['inputs']['corridors_mgra_xwalk_file']
carshare_mgra_file = config['inputs']['carshare_mgra_file']

In [13]:
# read config: parameters
population_density_threshold = config['parameters']['population_density_threshold']
high_density_carshare_participation = config['parameters']['high_density_mobility_hub_carshare_participation']
low_density_carshare_participation = config['parameters']['low_density_mobility_hub_carshare_participation']
college_carshare_participation = config['parameters']['college_carshare_participation']
military_carshare_participation = config['parameters']['military_carshare_participation']
base_year = config['parameters']['base_year']
scen_year = config['parameters']['scen_year']
daily_vmt_reduction_carshare = config['parameters']['daily_vmt_reduction_carshare']

In [14]:
# read config: outputs
output_dir = config['outputs']['output_dir']
output_results_filename = config['outputs']['output_file_name']

In [15]:
# read data
mgra_scen_input_df = pd.read_csv(mgra_scen_input_file)
mgra_base_input_df = pd.read_csv(mgra_base_input_file)
household_input_df = pd.read_csv(household_input_file)
person_input_df = pd.read_csv(person_input_file)
geo_xwalk_df = pd.read_csv(geography_xwalk_file)
emission_df = pd.read_excel(emission_factors_file)
corridor_mgra_df = pd.read_csv(corridor_mgra_xwalk_file)
carshare_mgra_df = pd.read_csv(carshare_mgra_file)
emission_df = pd.read_excel(emission_factors_file)

In [16]:
# preparing the main input dataset
adults_mgra_df = get_adult_population(person_input_df, household_input_df)
carshare_mgra_df = carshare_mgra_df[carshare_mgra_df.year == scen_year]

data_df = pd.merge(mgra_scen_input_df, adults_mgra_df, on = "mgra", how = "left")
data_df = pd.merge(data_df, carshare_mgra_df, left_on = "mgra", right_on = "mgra_13", how = "left")

data_df[["adult_pop", "MoHub_carshare_flag", "univ_flag", "MLB_flag"]] = data_df[["adult_pop", "MoHub_carshare_flag", "univ_flag", "MLB_flag"]].fillna(0)

data_df["pop_density"] = data_df["pop"] / data_df["acres"]
data_df["student_enrollment"] = data_df["collegeenroll"] + data_df["othercollegeenroll"]

data_df = data_df[["mgra", "pop", "adult_pop", "pop_density", "acres", "student_enrollment", "emp_total", "MoHub_carshare_flag", "univ_flag", "MLB_flag"]]

In [16]:
data_df.head()

,mgra,pop,adult_pop,pop_density,acres,student_enrollment,emp_total,MoHub_carshare_flag,univ_flag,MLB_flag
0,1,40,27.0,2.407399,16.615444,0,10,1.0,0.0,0.0
1,2,81,55.0,4.149763,19.519185,0,1,1.0,0.0,0.0
2,3,108,70.0,3.878597,27.845124,0,5,1.0,0.0,0.0
3,4,70,45.0,8.776133,7.976178,0,1,1.0,0.0,0.0
4,5,48,32.0,6.786849,7.072502,0,0,1.0,0.0,0.0


In [17]:
# build regional dataset with carshare market participations
regional_df = data_df.copy() 
regional_df = regional_df.assign(MoHub_carshare_participant = [low_density_carshare_participation if a <= population_density_threshold else high_density_carshare_participation for a in regional_df['pop_density']])

regional_df["mobility_hub"] = regional_df["adult_pop"] * regional_df["MoHub_carshare_participant"] * regional_df["MoHub_carshare_flag"] 
regional_df["college_staff"] = regional_df["emp_total"] * college_carshare_participation * regional_df["univ_flag"] 
regional_df["college_student"] = regional_df["student_enrollment"] * college_carshare_participation * regional_df["univ_flag"] 
regional_df["military_base"] = regional_df["emp_total"] * military_carshare_participation * regional_df["MLB_flag"] 
regional_df["total_carshare_market"] = regional_df["mobility_hub"] + regional_df["college_staff"] + regional_df["college_student"] + regional_df["military_base"]

regional_df = regional_df[["mgra", "pop", "mobility_hub", "college_staff", "college_student", "military_base", "total_carshare_market"]]

In [18]:
# build corridor level dataset with carshare market participations
corridor_df = pd.merge(regional_df, corridor_mgra_df, on = "mgra", how = "right")
corridor_df["mobility_hub"] = corridor_df["mobility_hub"] * corridor_df["weight"] 
corridor_df["college_staff"] = corridor_df["college_staff"] * corridor_df["weight"] 
corridor_df["college_student"] = corridor_df["college_student"] * corridor_df["weight"] 
corridor_df["military_base"] = corridor_df["military_base"] * corridor_df["weight"] 
corridor_df["total_carshare_market"] = corridor_df["total_carshare_market"] * corridor_df["weight"] 

corridor_df = corridor_df.groupby(["corridor"]).agg({"mobility_hub" : 'sum', "college_staff" : 'sum',
                                                    "college_student" : 'sum', "military_base" : 'sum', 
                                                    "total_carshare_market" : 'sum'})
corridor_df.reset_index(inplace = True)

In [19]:
# get emission factors for scenario year
emission_factors_df = get_emission_factors(emission_df, scen_year)

In [20]:
# compute regional emission reduction
regional_results_df = compute_regional_reductions(regional_df, emission_factors_df, daily_vmt_reduction_carshare, scen_year)
regional_results_df

,Variable,Value
0,Regional Population,3.620349e+06
1,Total Carshare Market,2.477635e+04
2,Total VMT Reduction by Carsharing,1.734344e+05
3,Daily Total GHG Reduction (short tons),4.076732e+01
4,Daily Per Capita GHG Reduction (lbs/person),2.252121e-02


In [21]:
# compute corridor level emission reduction
corridor_results_df = compute_corridor_reductions(corridor_df, emission_factors_df, daily_vmt_reduction_carshare, scen_year)
corridor_results_df

corridor,Variable,Central Mobility,Coast Canyons,North County,San Vicente,Sorrento
0,Scenario Year,2035.000000,2035.000000,2035.000000,2035.0,2035.000000
1,Total Carshare Market,3638.245000,2011.645808,4231.785000,0.0,7306.130000
2,Total Daily VMT Reduction,25467.715000,14081.520657,29622.495000,0.0,51142.910000
3,Daily Total GHG Reduction (short tons),5.986416,3.309988,6.963034,0.0,12.021601


In [23]:
# compute market segments emission reduction
market_segments_results_df = compute_market_segments_reductions(corridor_df, emission_factors_df, daily_vmt_reduction_carshare, scen_year)
market_segments_results_df

corridor,Variable,Central Mobility,Coast Canyons,North County,San Vicente,Sorrento
0,Scenario Year,2035.000000,2035.000000,2035.000000,2035.0,2035.000000
1,general population market,2700.205000,1279.145808,2409.445000,0.0,5357.210000
2,college staff market,103.060000,34.600000,211.340000,0.0,35.360000
3,college student market,834.980000,503.060000,1611.000000,0.0,731.600000
4,military market,0.000000,194.840000,0.000000,0.0,1181.960000
5,Total daily VMT reduction general population m...,18901.435000,8954.020657,16866.115000,0.0,37500.470000
6,Total daily VMT reduction college staff market,721.420000,242.200000,1479.380000,0.0,247.520000
7,Total daily VMT reduction college students market,5844.860000,3521.420000,11277.000000,0.0,5121.200000
8,Total daily VMT reduction military market,0.000000,1363.880000,0.000000,0.0,8273.720000
9,Daily Total GHG reduction general population m...,4.442952,2.104723,3.964532,0.0,8.814823


In [24]:
results_dict = {"Regional_Results": regional_results_df,
                "Corridor_Results": corridor_results_df,
                "Market_Segment_Results": market_segments_results_df,
                "Emission_Factors": emission_factors_df}

In [25]:
write_results(results_dict, output_results_filename, output_dir)